# 深度前馈网络
之所以叫前馈神经网络，是因为模型的输出与模型之间没有反馈连接，当前馈神经网络被扩展或包含反馈连接时，被称为**循环神经网络**。    
神经网络为非线性变换，但也可以理解为对线性变化加了一个非线性映射$\phi(x)$。传统的机器学习是选择一个通用的函数（如高斯，往往泛化效果不好）或者根据不同领域、不同模型人为设计，而深度学习通过数据去学习。    
此外在深度学习中线性函数可以立即为最后的输出层的仿射，而非线性映射是前面的隐藏层。    
线性模型和神经网络的最大区别在于神经网络的非线性使得**大多数代价函数变得非凸**。
## 6.2 基于梯度的学习    
1. 输出单元    
    基于最大似然条件估计，使用不同的分布，能得到不同的输出单元，如下：    
    * **用于高斯输出分布的线性单元**：基于仿射变换的线性输出单元，$\hat y = W^Th\ +\ b$，线性输出层经常被用来产生条件高斯分布的均值:$p(y|x) = N(y;\hat y, I)$，最大化其对数似然等价于最小化均方误差。    
    * **用于Bernoulli输出分布的sigmoid单元**：当需要预测二值型变量的值时，最大似然可以定义为y在x条件下的Bernoulli分布。    
    常用的函数为sigmoid,值得注意的是选用对数似然作为损失函数，正好能抵消sigmoid函数由指数计算带来的当z过大或者过小时的饱和问题。使得代价函数在z的预测错误且绝对值很大时依然能使用梯度下降法。其梯度为$|z|$。    
    * **用于Multinoulli输出分布的softmax单元**：和sigmoid一样，基于指数的softmax会饱和，适合用对数似然。     
    softmax是对输入差异之间的响应,softmax(z)=softmax(z+c)，所以导出softmax的稳定版本： $softmax(z-\max \limits_i z_i)$。    
    n个输出总和为1的约束意味着只需要n-1个参数，如sigmoid，所以我们用线性层的输出$z=W^Th+b$作为z是对分布的过度参数化，但是实践中，两者的差别很小，并且过度参数化的实现较为简单。    
    此外，softmax函数其实更接近于argmax，而不是max，“soft“的含义来源于softmax是连续可微的，可以当作argmax函数的软化版本。
 
## 6.3 隐藏单元
线性行为更容易优化的一般性原则（容易求导且稳定？但也不能完全线性），所以整流线性单元（relu)往往是隐藏单元的默认选择。    
整流线性单元不是在所有的输入点上都可微，在不可微的地方对于基于梯度的学习算法无效，但实践中梯度下降表现足够好的部分原因是：**神经网络训练算法通常不会达到代价函数的局部最小值，而是仅仅显著减小它的值**。    
1. 整流线性单元及其扩展     
relu的扩展有leaky relu和PReLU等。    
之前不知道的有**maxout**单元，将z划分为每组具有k个值的组，用来学习对输入x空间中多个方向响应的分段线性函数：     
&emsp; $g(z)_i = \max \limits_{j\in G(i)} z_j$      
其中每个$z_j$可以使用relu函数、leaky relu函数等。由于每个单元由多个字单元驱动，所以maxout具有一些冗余帮助它们抵抗**灾难性遗忘**。    
2. logistic sigmoid与双曲正切函数。  
sigmoid的饱和性使得其不被鼓励作为前馈网络中的隐藏单元，但是当必须使用sigmoid激活函数时，双曲正切函数通常比logistic sigmoid函数表现更好，因为以0为中心。    
sigmoid函数在前馈网络意外的情景中很常见，一些额外的要求使得其具有吸引力。    
3. 其他隐藏单元：    
很多其他的隐藏单元表现与传统隐藏单元相当，所以这不会引起兴趣，只有当新的隐藏单元类型被明确证明有显著改进时才会被发布。这里列举一些其他的典型的隐藏单元：    
    * 径向基函数：    
        &emsp;$ h_i = exp(-\frac{1}{\delta^2}\lVert W_{:,i} - x\rVert^2)$    
    但其对大部分x都饱和到0，因此很难被优化。    
    * softplus函数：$g(a) = \zeta (a) = log(1+e^a)$,整流线性单元的平滑版本，但根据经验，它并没有比relu表现更好。    
    * 硬双曲正切函数：g(a) = max(-1, min(1,a))    
    
## 6.4 架构设计
**架构**指网络的整体结构：应该具有多少单元，这些单元应该如何连接
**万能近似定理**表明，一个前馈神经网络如果具有线性输出层和至少一层具有任何一种“挤压”性质的激活函数的隐藏层，那么只要给予网络足够数量的隐藏单元，便可以以任意的精度近似任何从一个有限维空间到另一个有限维空间的Borel可测函数。只是不能保证训练算法能学习到这个函数，即前馈网络能学习到这个函数。    
此外具有单层的前馈网络足以表示任何函数，但网络层可能大的不可实现，并可能无法正确学习和泛化，于是在很多情况下，选用更深的模型。    
选择深度模型默许一个非常普遍的信念：我们想学得的函数应该设计几个更加简单的函数的组合。    